In [2]:
import pandas as pd
import numpy as np
import os

# Create folders
os.makedirs("raw", exist_ok=True)
os.makedirs("processed", exist_ok=True)

np.random.seed(42)

# Generate synthetic retail dataset
df = pd.DataFrame({
    "order_id": range(1, 201),
    "customer_id": np.random.randint(1000, 1100, 200),
    "product_id": np.random.randint(2000, 2100, 200),
    "order_date": pd.date_range(start="2024-01-01", periods=200, freq="D"),
    "sales": np.random.randint(100, 5000, 200),
    "profit": np.random.randint(10, 1000, 200)
})

# Intentionally add duplicates
df = pd.concat([df, df.iloc[:5]])

# Save raw data
df.to_csv("raw/retail.csv", index=False)

df.head()


,order_id,customer_id,product_id,order_date,sales,profit
0,1,1051,2008,2024-01-01,3109,804
1,2,1092,2061,2024-01-02,254,643
2,3,1014,2036,2024-01-03,2637,203
3,4,1071,2096,2024-01-04,1509,526
4,5,1060,2050,2024-01-05,4157,38


In [3]:
df = pd.read_csv("raw/retail.csv")
print("Original rows:", len(df))


Original rows: 205


In [4]:
df = df.drop_duplicates()
print("After removing duplicates:", len(df))


After removing duplicates: 200


In [5]:
df = df.fillna(0)


In [7]:
df.columns = df.columns.str.lower().str.strip().str.replace(" ", "_")


In [8]:
df['order_date'] = pd.to_datetime(df['order_date'])
df['sales'] = df['sales'].astype(float)
df['profit'] = df['profit'].astype(float)


In [14]:
customers = df[['customer_id']].drop_duplicates()

orders = df[['order_id', 'order_date', 'customer_id', 'sales', 'profit', 'profit_margin']]

products = df[['product_id']].drop_duplicates()


In [10]:
print(df.columns)


Index(['order_id', 'customer_id', 'product_id', 'order_date', 'sales',
       'profit'],
      dtype='object')


In [15]:
df.to_csv("processed/processed_data.csv", index=False)
customers.to_csv("processed/customers.csv", index=False)
orders.to_csv("processed/orders.csv", index=False)
products.to_csv("processed/products.csv", index=False)


In [11]:
# Create derived column
df['profit_margin'] = df['profit'] / df['sales']


In [12]:
print(df.columns)


Index(['order_id', 'customer_id', 'product_id', 'order_date', 'sales',
       'profit', 'profit_margin'],
      dtype='object')


In [13]:
orders = df[['order_id',
             'order_date',
             'customer_id',
             'sales',
             'profit',
             'profit_margin']]


In [16]:
import sqlite3

conn = sqlite3.connect("database.sqlite")

customers.to_sql("customers", conn, if_exists="replace", index=False)
orders.to_sql("orders", conn, if_exists="replace", index=False)
products.to_sql("products", conn, if_exists="replace", index=False)

conn.close()


In [17]:
print("Final row count:", len(df))
print("Null values:\n", df.isnull().sum())


Final row count: 200
Null values:
 order_id         0
customer_id      0
product_id       0
order_date       0
sales            0
profit           0
profit_margin    0
dtype: int64
